In [286]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch
import numpy as np

In [287]:
data = pd.read_csv("2025-04-08/data_1.csv")
data

,x,y,RSSI_Sensor_1,RSSI_Sensor_2,RSSI_Sensor_3,RSSI_Sensor_4,Avg_RSSI
0,0.000000,0.0,-40.000000,-58.061800,-62.577250,-58.061800,-54.675212
1,0.444444,0.0,-29.434524,-58.141732,-61.855044,-56.527224,-51.489631
2,0.888889,0.0,-38.465424,-58.375808,-61.143675,-54.787466,-53.193093
3,1.333333,0.0,-43.748162,-58.748162,-60.457312,-52.779062,-53.933175
4,1.777778,0.0,-47.496324,-59.236100,-59.814112,-50.403625,-54.237540
...,...,...,...,...,...,...,...
95,2.222222,4.0,-59.814112,-50.403625,-47.496324,-59.236100,-54.237540
96,2.666667,4.0,-60.457312,-52.779062,-43.748162,-58.748162,-53.933175
97,3.111111,4.0,-61.143675,-54.787466,-38.465424,-58.375808,-53.193093
98,3.555556,4.0,-61.855044,-56.527224,-29.434524,-58.141732,-51.489631


In [288]:
X = data[["RSSI_Sensor_1", "RSSI_Sensor_2", "RSSI_Sensor_3", "RSSI_Sensor_4"]]
y = data[["x", "y"]]

scaler_X = StandardScaler().fit(X)
scaler_y = StandardScaler().fit(y)

X_scaled = scaler_X.transform(X)
y_scaled = scaler_y.transform(y)

In [289]:
class RSSIModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.LayerNorm(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        return self.layers(x)


In [290]:
dataset = TensorDataset(torch.FloatTensor(X_scaled),
                       torch.FloatTensor(y_scaled))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)

model = RSSIModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [291]:
for epoch in range(50):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')


Epoch 0, Loss: 1.1034
Epoch 0, Loss: 0.8490
Epoch 0, Loss: 0.9280
Epoch 1, Loss: 0.6244
Epoch 1, Loss: 0.5446
Epoch 1, Loss: 0.3880
Epoch 2, Loss: 0.2343
Epoch 2, Loss: 0.2355
Epoch 2, Loss: 0.2286
Epoch 3, Loss: 0.0813
Epoch 3, Loss: 0.1182
Epoch 3, Loss: 0.1210
Epoch 4, Loss: 0.0822
Epoch 4, Loss: 0.0803
Epoch 4, Loss: 0.1071
Epoch 5, Loss: 0.1028
Epoch 5, Loss: 0.1140
Epoch 5, Loss: 0.1173
Epoch 6, Loss: 0.1160
Epoch 6, Loss: 0.1144
Epoch 6, Loss: 0.1255
Epoch 7, Loss: 0.1133
Epoch 7, Loss: 0.0918
Epoch 7, Loss: 0.0812
Epoch 8, Loss: 0.0814
Epoch 8, Loss: 0.0631
Epoch 8, Loss: 0.0586
Epoch 9, Loss: 0.0586
Epoch 9, Loss: 0.0502
Epoch 9, Loss: 0.0438
Epoch 10, Loss: 0.0449
Epoch 10, Loss: 0.0513
Epoch 10, Loss: 0.0500
Epoch 11, Loss: 0.0517
Epoch 11, Loss: 0.0507
Epoch 11, Loss: 0.0479
Epoch 12, Loss: 0.0371
Epoch 12, Loss: 0.0625
Epoch 12, Loss: 0.0345
Epoch 13, Loss: 0.0562
Epoch 13, Loss: 0.0282
Epoch 13, Loss: 0.0334
Epoch 14, Loss: 0.0289
Epoch 14, Loss: 0.0433
Epoch 14, Loss: 0.

In [292]:
with torch.no_grad():
    test_inputs = torch.FloatTensor(X_scaled)
    predictions = model(test_inputs)
    mse = criterion(predictions, torch.FloatTensor(y_scaled))
    print(f'Final MSE: {mse.item():.4f}')


Final MSE: 0.0051


In [293]:
predicted_coords = scaler_y.inverse_transform(predictions.numpy())
predicted_coords

array([[0.27264404, 0.47636175],
       [0.40536487, 0.20326483],
       [0.7884047 , 0.14070499],
       [1.293721  , 0.15028548],
       [1.7243067 , 0.08904409],
       [2.2305608 , 0.05407524],
       [2.6823697 , 0.08831751],
       [3.1454744 , 0.06402802],
       [3.5927427 , 0.1538527 ],
       [3.8784685 , 0.34106767],
       [0.1780895 , 0.41148663],
       [0.3813938 , 0.36563528],
       [0.7912959 , 0.33300233],
       [1.3221567 , 0.379135  ],
       [1.7373325 , 0.3560295 ],
       [2.2151082 , 0.37657368],
       [2.6549706 , 0.36750138],
       [3.1520848 , 0.31504345],
       [3.6042013 , 0.3365699 ],
       [3.8487961 , 0.38212085],
       [0.07382822, 0.78287685],
       [0.3462447 , 0.7328267 ],
       [0.8018658 , 0.72162366],
       [1.3642707 , 0.7845079 ],
       [1.7884778 , 0.8486452 ],
       [2.2161055 , 0.90366983],
       [2.647509  , 0.84207237],
       [3.1481066 , 0.8008268 ],
       [3.63393   , 0.7941326 ],
       [3.9765823 , 0.8340094 ],
       [0.

In [308]:
# Пример входных данных
input_rssi = np.array([[-58.45673382067411, -60.96910013008056, -54.31363764158987, -40.0]])

# Создание DataFrame
input_df = pd.DataFrame(
    input_rssi,
    columns=['RSSI_Sensor_1', 'RSSI_Sensor_2', 'RSSI_Sensor_3', 'RSSI_Sensor_4']
)

# Нормализация
input_scaled = scaler_X.transform(input_df)

# Прогнозирование
input_tensor = torch.FloatTensor(input_scaled)
with torch.no_grad():
    predicted = model(input_tensor)

# Обратное преобразование
coords = scaler_y.inverse_transform(predicted.numpy())

print(f"Координаты: x={coords[0][0]:.2f}, y={coords[0][1]:.2f}")

Координаты: x=3.99, y=0.97


In [305]:
# Загрузка данных (указать правильный путь)
data = pd.read_csv('tests/data_3.csv')

# Разделение на признаки и целевую переменную
X = data[['RSSI_Sensor_1', 'RSSI_Sensor_2', 'RSSI_Sensor_3', 'RSSI_Sensor_4']]
y_actual = data[['x', 'y']]

# Нормализация данных (предполагается, что scaler_X и scaler_y уже обучены)
X_scaled = scaler_X.transform(X)

# Прогнозирование
model.eval()
with torch.no_grad():
    predictions = model(torch.FloatTensor(X_scaled))

# Обратное преобразование координат
y_pred = scaler_y.inverse_transform(predictions.numpy())

# Сравнение результатов
results = pd.DataFrame({
    'X_pred': y_pred[:,0],
    'Y_pred': y_pred[:,1],
    'X_actual': y_actual['x'],
    'Y_actual': y_actual['y']
})

# Расчет ошибки
results['X_error'] = abs(results['X_pred'] - results['X_actual'])
results['Y_error'] = abs(results['Y_pred'] - results['Y_actual'])
print(results)

# Вычисление средних ошибок
mean_x_error = round(results['X_error'].mean(), 2)
mean_y_error = round(results['Y_error'].mean(), 2)

print(f"\nСредняя ошибка по X: {mean_x_error} мм")
print(f"Средняя ошибка по Y: {mean_y_error} мм")



      X_pred    Y_pred  X_actual  Y_actual   X_error   Y_error
0   0.043675  3.081597       0.0       3.0  0.043675  0.081597
1   0.060611  1.976142       0.0       2.0  0.060611  0.023858
2   0.902996  0.144719       1.0       0.0  0.097004  0.144719
3   1.984356  0.069513       2.0       0.0  0.015644  0.069513
4   3.009348  0.056764       3.0       0.0  0.009348  0.056764
5   3.991958  0.970766       4.0       1.0  0.008042  0.029234
6   3.952580  2.038964       4.0       2.0  0.047420  0.038964
7   0.910686  3.949945       1.0       4.0  0.089314  0.050055
8   3.052644  3.612997       3.0       3.5  0.052644  0.112997
9   3.052644  3.612997       3.0       3.5  0.052644  0.112997
10  1.502903  2.928355       1.5       3.0  0.002903  0.071645
11  2.042225  2.996384       2.0       3.0  0.042225  0.003616
12  1.572991  2.087716       1.5       2.0  0.072991  0.087716
13  3.052644  3.612997       3.0       3.5  0.052644  0.112997
14  2.016834  1.023825       2.0       1.0  0.016834  0